In [14]:
! pip install tqdm


Fetching shorts from @BrajaBeats...
Error fetching channel shorts: Expecting value: line 1 column 1 (char 0)


In [16]:
!pip install yt-dlp

In [17]:
!pip install speech_recognition librosa numpy pydub fuzzzywuzzy

ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [18]:
!pip install  pygame

In [1]:
import tkinter as tk
from tkinter import filedialog, ttk, scrolledtext
import json
import pygame
import time
import os
import re
import glob

class GitaManualTagger:
    def __init__(self, root):
        self.root = root
        self.root.title("Gita Audio Tagger")
        self.root.geometry("1200x800")  # Increased window size for better visibility
        
        # Initialize pygame for audio
        pygame.init()
        pygame.mixer.init()
        
        self.audio_file = None
        self.audio_files = []  # To store available audio files
        self.current_audio_index = 0
        self.verse_data = None
        self.all_verses = []  # To store all verse data from gita.json
        self.current_verse_index = 0
        self.current_word = None
        self.playing = False
        self.current_position = 0
        self.gita_json_path = None
        
        # Setup UI
        self.setup_ui()
        
        # Auto-load gita.json and audio files if available
        self.auto_load_resources()
    
    def auto_load_resources(self):
        # Use current working directory instead of __file__
        current_dir = os.getcwd()

        # Try to load gita.json
        gita_json_path = os.path.join(current_dir, "gita.json")
        if os.path.exists(gita_json_path):
            self.load_gita_json_file(gita_json_path)
            self.status_var.set(f"Automatically loaded gita.json with {len(self.all_verses)} verses")
        
        # Try to load audio files from BrajaBeats_Gita_MP3 folder
        audio_dir = os.path.join(current_dir, "BrajaBeats_Gita_MP3")
        if os.path.exists(audio_dir) and os.path.isdir(audio_dir):
            self.load_audio_directory_path(audio_dir)
    
    def setup_ui(self):
        # Main frames
        left_frame = tk.Frame(self.root, width=600)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        right_frame = tk.Frame(self.root, width=600)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Audio Files Section (Left Frame Top)
        audio_files_frame = tk.LabelFrame(left_frame, text="Audio Files")
        audio_files_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Audio controls
        audio_control_frame = tk.Frame(audio_files_frame)
        audio_control_frame.pack(fill=tk.X, pady=5)
        
        tk.Button(audio_control_frame, text="Load Audio Directory", command=self.load_audio_directory).pack(side=tk.LEFT, padx=5)
        tk.Button(audio_control_frame, text="Play/Pause", command=self.toggle_play).pack(side=tk.LEFT, padx=5)
        tk.Button(audio_control_frame, text="Previous", command=self.previous_audio).pack(side=tk.LEFT, padx=5)
        tk.Button(audio_control_frame, text="Next", command=self.next_audio).pack(side=tk.LEFT, padx=5)
        
        self.position_label = tk.Label(audio_control_frame, text="Position: 0:00")
        self.position_label.pack(side=tk.LEFT, padx=20)
        
        # Audio filter
        filter_frame = tk.Frame(audio_files_frame)
        filter_frame.pack(fill=tk.X, pady=5)
        
        tk.Label(filter_frame, text="Filter by Chapter:").pack(side=tk.LEFT, padx=5)
        self.chapter_filter_var = tk.StringVar()
        self.chapter_filter_var.trace('w', self.filter_audio_files)
        self.chapter_filter = tk.Entry(filter_frame, textvariable=self.chapter_filter_var, width=5)
        self.chapter_filter.pack(side=tk.LEFT, padx=5)
        
        tk.Label(filter_frame, text="Verse:").pack(side=tk.LEFT, padx=5)
        self.verse_filter_var = tk.StringVar()
        self.verse_filter_var.trace('w', self.filter_audio_files)
        self.verse_filter = tk.Entry(filter_frame, textvariable=self.verse_filter_var, width=5)
        self.verse_filter.pack(side=tk.LEFT, padx=5)
        
        tk.Button(filter_frame, text="Clear Filters", command=self.clear_filters).pack(side=tk.LEFT, padx=20)
        
        # Audio files listbox with scrollbar
        listbox_frame = tk.Frame(audio_files_frame)
        listbox_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        scrollbar = tk.Scrollbar(listbox_frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.audio_listbox = tk.Listbox(listbox_frame, height=10, yscrollcommand=scrollbar.set)
        self.audio_listbox.pack(fill=tk.BOTH, expand=True)
        self.audio_listbox.bind('<<ListboxSelect>>', self.on_audio_select)
        scrollbar.config(command=self.audio_listbox.yview)
        
        # Current audio info
        self.current_audio_info = tk.Label(audio_files_frame, text="No audio loaded", wraplength=550, justify=tk.LEFT)
        self.current_audio_info.pack(fill=tk.X, pady=5)
        
        # Verse Data Section (Right Frame Top)
        verse_frame = tk.LabelFrame(right_frame, text="Gita Verse Data")
        verse_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Verse controls
        verse_control_frame = tk.Frame(verse_frame)
        verse_control_frame.pack(fill=tk.X, pady=5)
        
        tk.Button(verse_control_frame, text="Load Gita JSON", command=self.load_gita_json).pack(side=tk.LEFT, padx=5)
        tk.Button(verse_control_frame, text="Previous Verse", command=self.previous_verse).pack(side=tk.LEFT, padx=5)
        tk.Button(verse_control_frame, text="Next Verse", command=self.next_verse).pack(side=tk.LEFT, padx=5)
        tk.Button(verse_control_frame, text="Extract Chapter/Verse", command=self.extract_from_audio_filename).pack(side=tk.LEFT, padx=5)
        
        # Verse navigation
        nav_frame = tk.Frame(verse_frame)
        nav_frame.pack(fill=tk.X, pady=5)
        
        tk.Label(nav_frame, text="Chapter:").pack(side=tk.LEFT, padx=2)
        self.chapter_var = tk.StringVar()
        self.chapter_entry = tk.Entry(nav_frame, textvariable=self.chapter_var, width=5)
        self.chapter_entry.pack(side=tk.LEFT, padx=2)
        
        tk.Label(nav_frame, text="Verse:").pack(side=tk.LEFT, padx=2)
        self.verse_var = tk.StringVar()
        self.verse_entry = tk.Entry(nav_frame, textvariable=self.verse_var, width=5)
        self.verse_entry.pack(side=tk.LEFT, padx=2)
        
        tk.Button(nav_frame, text="Go", command=self.go_to_verse).pack(side=tk.LEFT, padx=5)
        
        # Verse details
        self.verse_display = scrolledtext.ScrolledText(verse_frame, height=5, wrap=tk.WORD)
        self.verse_display.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Word Tagging Section (Bottom Frame spans both)
        tagging_frame = tk.LabelFrame(self.root, text="Word Tagging")
        tagging_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)
        
        # Word selection panel - make it a two-panel layout
        word_frame = tk.Frame(tagging_frame)
        word_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Word list with scrollbar on the left
        word_list_frame = tk.Frame(word_frame)
        word_list_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)
        
        tk.Label(word_list_frame, text="Sanskrit Words:").pack(anchor=tk.W)
        
        word_scrollbar = tk.Scrollbar(word_list_frame)
        word_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.word_listbox = tk.Listbox(word_list_frame, height=8, yscrollcommand=word_scrollbar.set)
        self.word_listbox.pack(fill=tk.BOTH, expand=True, pady=5)
        self.word_listbox.bind('<<ListboxSelect>>', self.on_word_select)
        word_scrollbar.config(command=self.word_listbox.yview)
        
        # Word details on the right
        word_details_frame = tk.Frame(word_frame)
        word_details_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5)
        
        tk.Label(word_details_frame, text="Word Details:").pack(anchor=tk.W)
        self.word_details = scrolledtext.ScrolledText(word_details_frame, height=8, wrap=tk.WORD)
        self.word_details.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Timestamp controls
        ts_frame = tk.Frame(tagging_frame)
        ts_frame.pack(fill=tk.X, pady=5)
        
        tk.Button(ts_frame, text="Mark Start", command=lambda: self.mark_timestamp("start")).pack(side=tk.LEFT, padx=5)
        tk.Button(ts_frame, text="Mark End", command=lambda: self.mark_timestamp("end")).pack(side=tk.LEFT, padx=5)
        tk.Button(ts_frame, text="Play Word", command=self.play_current_word_segment).pack(side=tk.LEFT, padx=5)
        tk.Button(ts_frame, text="Jump to Start", command=lambda: self.jump_to_timestamp("start")).pack(side=tk.LEFT, padx=5)
        tk.Button(ts_frame, text="Jump to End", command=lambda: self.jump_to_timestamp("end")).pack(side=tk.LEFT, padx=5)
        
        # Save button
        tk.Button(tagging_frame, text="Save Tagged Data", command=self.save_tagged, bg="#4CAF50", fg="white").pack(pady=10)
        
        # Status bar
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        status_bar = tk.Label(self.root, textvariable=self.status_var, bd=1, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Start position update timer
        self.update_position()
    
    def load_gita_json(self):
        json_file = filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])
        if json_file:
            self.load_gita_json_file(json_file)
    
    def load_gita_json_file(self, json_file):
        self.gita_json_path = json_file
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                self.all_verses = json.load(f)
            
            # Create a dictionary for quick lookup by chapter and verse
            self.verse_lookup = {}
            for i, verse in enumerate(self.all_verses):
                if 'chapter' in verse and 'shloka' in verse:
                    key = f"{verse['chapter']}.{verse['shloka']}"
                    self.verse_lookup[key] = i
            
            self.status_var.set(f"Loaded {len(self.all_verses)} verses from Gita JSON")
            self.current_verse_index = 0
            self.display_current_verse()
            
        except Exception as e:
            self.status_var.set(f"Error loading Gita JSON: {str(e)}")
    
    def display_current_verse(self):
        if not self.all_verses or self.current_verse_index >= len(self.all_verses):
            return
        
        self.verse_data = self.all_verses[self.current_verse_index]
        
        # Update chapter/verse entries
        if 'chapter' in self.verse_data:
            self.chapter_var.set(self.verse_data['chapter'])
        if 'shloka' in self.verse_data:
            self.verse_var.set(self.verse_data['shloka'])
        
        # Clear and populate word list
        self.word_listbox.delete(0, tk.END)
        if 'synonyms' in self.verse_data:
            for word in self.verse_data['synonyms']:
                self.word_listbox.insert(tk.END, word)
        
        # Update verse display
        self.verse_display.delete(1.0, tk.END)
        
        if 'sanskrit' in self.verse_data:
            self.verse_display.insert(tk.END, f"Sanskrit:\n{self.verse_data['sanskrit']}\n\n")
        
        if 'english' in self.verse_data:
            self.verse_display.insert(tk.END, f"English Transliteration:\n{self.verse_data['english']}\n\n")
            
        if 'translation' in self.verse_data:
            self.verse_display.insert(tk.END, f"Translation:\n{self.verse_data['translation']}\n\n")
        
        self.status_var.set(f"Displaying Verse {self.current_verse_index+1}/{len(self.all_verses)}: Chapter {self.verse_data.get('chapter', '?')}, Verse {self.verse_data.get('shloka', '?')}")
        
        # Try to find and select the corresponding audio file
        self.find_corresponding_audio()
    
    def find_corresponding_audio(self):
        if not self.audio_files or not self.verse_data:
            return
        
        chapter = self.verse_data.get('chapter')
        verse = self.verse_data.get('shloka')
        
        if not chapter or not verse:
            return
        
        # Look for a matching audio file
        pattern = f"Bhagavad-gita {chapter}.{verse}"
        for i, audio_file in enumerate(self.audio_files):
            filename = os.path.basename(audio_file)
            if pattern in filename:
                self.current_audio_index = i
                self.load_audio(audio_file)
                self.audio_listbox.selection_clear(0, tk.END)
                self.audio_listbox.selection_set(i)
                self.audio_listbox.see(i)
                self.status_var.set(f"Found matching audio file for Chapter {chapter}, Verse {verse}")
                return
        
        self.status_var.set(f"No matching audio found for Chapter {chapter}, Verse {verse}")
    
    def go_to_verse(self):
        try:
            chapter = self.chapter_var.get()
            verse = self.verse_var.get()
            
            if not chapter or not verse:
                self.status_var.set("Please enter both chapter and verse numbers")
                return
            
            # Try direct lookup first
            key = f"{chapter}.{verse}"
            if key in self.verse_lookup:
                self.current_verse_index = self.verse_lookup[key]
                self.display_current_verse()
                return
            
            # Fallback to linear search
            for i, verse_data in enumerate(self.all_verses):
                if str(verse_data.get('chapter', '')) == chapter and str(verse_data.get('shloka', '')) == verse:
                    self.current_verse_index = i
                    self.display_current_verse()
                    return
            
            self.status_var.set(f"Chapter {chapter}, Verse {verse} not found")
            
        except Exception as e:
            self.status_var.set(f"Error navigating to verse: {str(e)}")
    
    def next_verse(self):
        if self.all_verses and self.current_verse_index < len(self.all_verses) - 1:
            self.current_verse_index += 1
            self.display_current_verse()
    
    def previous_verse(self):
        if self.all_verses and self.current_verse_index > 0:
            self.current_verse_index -= 1
            self.display_current_verse()
    
    def load_audio_directory(self):
        audio_dir = filedialog.askdirectory(title="Select Directory with Gita MP3 Files")
        if audio_dir:
            self.load_audio_directory_path(audio_dir)
    
    def load_audio_directory_path(self, audio_dir):
        # Find all MP3 files
        self.audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) 
                            if f.lower().endswith('.mp3') and 'bhagavad-gita' in f.lower()]
        
        # Sort by chapter and verse if possible
        self.audio_files.sort(key=self.sort_by_chapter_verse)
        
        # Clear and populate audio list
        self.audio_listbox.delete(0, tk.END)
        for f in self.audio_files:
            self.audio_listbox.insert(tk.END, os.path.basename(f))
        
        self.status_var.set(f"Loaded {len(self.audio_files)} Gita audio files")
        
        # Load the first audio file if available
        if self.audio_files:
            self.current_audio_index = 0
            self.load_audio(self.audio_files[0])
    
    def sort_by_chapter_verse(self, file_path):

        filename = os.path.basename(file_path)
        match = re.search(r'Bhagavad-gita\s+(\d+)\.(\d+)', filename)
        if match:
            chapter = int(match.group(1))
            verse = int(match.group(2))
            return (chapter, verse)
        return (999, 999)  # Default high value for files that don't match pattern
    
    def filter_audio_files(self, *args):
        if not self.audio_files:
            return
        
        chapter_filter = self.chapter_filter_var.get().strip()
        verse_filter = self.verse_filter_var.get().strip()
        
        # Clear the listbox
        self.audio_listbox.delete(0, tk.END)
        
        # Apply filters
        for f in self.audio_files:
            filename = os.path.basename(f)
            match = re.search(r'Bhagavad-gita\s+(\d+)\.(\d+)', filename)
            
            if match:
                chapter = match.group(1)
                verse = match.group(2)
                
                # Check if it matches the filters
                chapter_match = not chapter_filter or chapter_filter == chapter
                verse_match = not verse_filter or verse_filter == verse
                
                if chapter_match and verse_match:
                    self.audio_listbox.insert(tk.END, filename)
        
        self.status_var.set(f"Filtered: {self.audio_listbox.size()} audio files displayed")
    
    def clear_filters(self):
        self.chapter_filter_var.set("")
        self.verse_filter_var.set("")
        
        # Reload all audio files
        self.audio_listbox.delete(0, tk.END)
        for f in self.audio_files:
            self.audio_listbox.insert(tk.END, os.path.basename(f))
        
        self.status_var.set(f"Filters cleared: {len(self.audio_files)} audio files displayed")
    
    def load_audio(self, audio_path):
        try:
            pygame.mixer.music.load(audio_path)
            self.audio_file = audio_path
            self.current_audio_info.config(text=f"Current Audio: {os.path.basename(audio_path)}")
            self.status_var.set(f"Loaded audio: {os.path.basename(audio_path)}")
            
            # Try to extract chapter and verse from filename and load the corresponding verse
            self.extract_from_audio_filename()
            
        except Exception as e:
            self.status_var.set(f"Error loading audio: {str(e)}")
    
    def extract_from_audio_filename(self):
        if not self.audio_file:
            self.status_var.set("No audio file loaded")
            return
        
        filename = os.path.basename(self.audio_file)
        # Try to extract chapter and verse using regex
        match = re.search(r'Bhagavad-gita\s+(\d+)\.(\d+)', filename)
        if match:
            chapter = match.group(1)
            verse = match.group(2)
            
            self.chapter_var.set(chapter)
            self.verse_var.set(verse)
            self.go_to_verse()
            self.status_var.set(f"Extracted Chapter {chapter}, Verse {verse} from filename")
        else:
            self.status_var.set("Could not extract chapter and verse from filename")
    
    def on_audio_select(self, event):
        selection = self.audio_listbox.curselection()
        if selection:
            index = selection[0]
            filename = self.audio_listbox.get(index)
            
            # Find the full path of the selected file
            for f in self.audio_files:
                if os.path.basename(f) == filename:
                    self.current_audio_index = self.audio_files.index(f)
                    self.load_audio(f)
                    break
    
    def next_audio(self):
        if self.audio_files and self.current_audio_index < len(self.audio_files) - 1:
            self.current_audio_index += 1
            self.load_audio(self.audio_files[self.current_audio_index])
            
            # Update selection in listbox
            self.audio_listbox.selection_clear(0, tk.END)
            
            # Find the index in the filtered listbox
            filename = os.path.basename(self.audio_files[self.current_audio_index])
            for i in range(self.audio_listbox.size()):
                if self.audio_listbox.get(i) == filename:
                    self.audio_listbox.selection_set(i)
                    self.audio_listbox.see(i)
                    break
    
    def previous_audio(self):
        if self.audio_files and self.current_audio_index > 0:
            self.current_audio_index -= 1
            self.load_audio(self.audio_files[self.current_audio_index])
            
            # Update selection in listbox
            self.audio_listbox.selection_clear(0, tk.END)
            
            # Find the index in the filtered listbox
            filename = os.path.basename(self.audio_files[self.current_audio_index])
            for i in range(self.audio_listbox.size()):
                if self.audio_listbox.get(i) == filename:
                    self.audio_listbox.selection_set(i)
                    self.audio_listbox.see(i)
                    break
    
    def toggle_play(self):
        if not self.audio_file:
            self.status_var.set("No audio file loaded")
            return
            
        if self.playing:
            print("Pausing audio")
            try:
                pygame.mixer.music.pause()
                print("Pause command sent to pygame")
            except Exception as e:
                print(f"Error pausing audio: {e}")
                # Try to reinitialize pygame mixer
                try:
                    pygame.mixer.quit()
                    pygame.mixer.init()
                    if self.audio_file:
                        pygame.mixer.music.load(self.audio_file)
                        print("Reinitialized pygame mixer and reloaded audio")
                except Exception as re:
                    print(f"Failed to reinitialize mixer: {re}")
            self.playing = False
            self.status_var.set("Audio paused")
        else:
            print("Playing audio")
            try:
                # First check if audio file is properly loaded
                if not self.audio_file:
                    raise ValueError("No audio file loaded")
                    
                # Try playing or unpausing based on current state
                if not pygame.mixer.get_init():
                    pygame.mixer.init()
                    pygame.mixer.music.load(self.audio_file)
                    
                if pygame.mixer.music.get_pos() <= 0:  # Not started or stopped
                    pygame.mixer.music.load(self.audio_file)  # Reload to ensure it works
                    pygame.mixer.music.play()
                    print("Starting playback from beginning")
                else:
                    pygame.mixer.music.unpause()
                    print(f"Unpausing at position {pygame.mixer.music.get_pos()}ms")
                
                self.playing = True
                self.status_var.set("Audio playing")
                
            except Exception as e:
                print(f"Playback error: {e}")
                # Try to recover by reinitializing
                try:
                    pygame.mixer.quit()
                    pygame.mixer.init()
                    if self.audio_file:
                        pygame.mixer.music.load(self.audio_file)
                        pygame.mixer.music.play()
                        self.playing = True
                        self.status_var.set("Audio playing (after recovery)")
                except Exception as re:
                    self.status_var.set(f"Cannot play audio: {str(re)}")
    
    def update_position(self):
        if self.audio_file and pygame.mixer.music.get_busy():
            pos_ms = pygame.mixer.music.get_pos()
            if pos_ms >= 0:
                self.current_position = pos_ms
                minutes = pos_ms // 60000
                seconds = (pos_ms % 60000) // 1000
                milliseconds = pos_ms % 1000
                self.position_label.config(text=f"Position: {minutes}:{seconds:02d}.{milliseconds:03d}")
        
        # Schedule next update
        self.root.after(50, self.update_position)
    
    def on_word_select(self, event):
        if not self.verse_data:
            return
            
        selection = self.word_listbox.curselection()
        if selection:
            word_index = selection[0]
            word_text = self.word_listbox.get(word_index)
            self.current_word = word_text
            
            if 'synonyms' in self.verse_data and word_text in self.verse_data['synonyms']:
                word_data = self.verse_data['synonyms'][word_text]
                
                # Update word details
                self.word_details.delete(1.0, tk.END)
                details = f"Sanskrit: {word_text}\n"
                
                if 'meaning' in word_data:
                    details += f"Meaning: {word_data['meaning']}\n"
                
                if 'versetext' in word_data:
                    details += f"Verse text: {word_data['versetext']}\n"
                
                # Add timestamp info if available
                if 'timestamp' in word_data:
                    start = word_data['timestamp'].get('start')
                    end = word_data['timestamp'].get('end')
                    if start is not None:
                        start_min = start // 60000
                        start_sec = (start % 60000) // 1000
                        start_ms = start % 1000
                        details += f"Start: {start_min}:{start_sec:02d}.{start_ms:03d}\n"
                    else:
                        details += "Start: Not marked\n"
                        
                    if end is not None:
                        end_min = end // 60000
                        end_sec = (end % 60000) // 1000
                        end_ms = end % 1000
                        details += f"End: {end_min}:{end_sec:02d}.{end_ms:03d}\n"
                    else:
                        details += "End: Not marked\n"
                else:
                    details += "No timestamps marked yet"
                
                self.word_details.insert(tk.END, details)
    
    def mark_timestamp(self, mark_type):
        if not self.audio_file or not self.current_word or not self.verse_data:
            self.status_var.set("Cannot mark timestamp: Ensure audio file is loaded and a word is selected")
            return
        
        if 'synonyms' not in self.verse_data or self.current_word not in self.verse_data['synonyms']:
            self.status_var.set(f"Word '{self.current_word}' not found in verse data")
            return
        
        # Ensure timestamp field exists
        if 'timestamp' not in self.verse_data['synonyms'][self.current_word]:
            self.verse_data['synonyms'][self.current_word]['timestamp'] = {"start": None, "end": None}
        
        # Set the timestamp
        self.verse_data['synonyms'][self.current_word]['timestamp'][mark_type] = self.current_position
        
        # Update display
        self.on_word_select(None)  # Refresh details
        
        min_pos = self.current_position // 60000
        sec_pos = (self.current_position % 60000) // 1000
        ms_pos = self.current_position % 1000
        time_str = f"{min_pos}:{sec_pos:02d}.{ms_pos:03d}"
        
        self.status_var.set(f"Marked {mark_type} timestamp for '{self.current_word}' at {time_str}")
    
    def jump_to_timestamp(self, mark_type):
        if not self.audio_file or not self.current_word or not self.verse_data:
            self.status_var.set("Cannot jump: Ensure audio file is loaded and a word is selected")
            return
        
        try:
            timestamp = self.verse_data['synonyms'][self.current_word]['timestamp'][mark_type]
            if timestamp is not None:
                # Convert to seconds for pygame
                timestamp_sec = timestamp / 1000
                pygame.mixer.music.play(start=timestamp_sec)
                self.playing = True
                self.status_var.set(f"Jumped to {mark_type} timestamp of '{self.current_word}'")
            else:
                self.status_var.set(f"No {mark_type} timestamp set for '{self.current_word}'")
        except (KeyError, TypeError):
            self.status_var.set(f"No {mark_type} timestamp available for '{self.current_word}'")
    
    def play_current_word_segment(self):
        if not self.audio_file or not self.current_word or not self.verse_data:
            self.status_var.set("Cannot play segment: Ensure audio file is loaded and a word is selected")
            return
        
        try:
            timestamps = self.verse_data['synonyms'][self.current_word]['timestamp']
            start = timestamps.get('start')
            end = timestamps.get('end')
            
            if start is not None and end is not None:
                # Convert to seconds for pygame
                start_sec = start / 1000
                pygame.mixer.music.play(start=start_sec)
                
                # Calculate duration
                duration = (end - start) / 1000
                
                # Schedule stop after duration
                self.root.after(int(duration * 1000), self.stop_playback)
                
                self.playing = True
                self.status_var.set(f"Playing segment for '{self.current_word}'")
            else:
                self.status_var.set(f"Both start and end timestamps must be set for '{self.current_word}'")
        except (KeyError, TypeError):
            self.status_var.set(f"Timestamps not properly set for '{self.current_word}'")
    
    def stop_playback(self):
        if self.playing:
            pygame.mixer.music.pause()
            self.playing = False
    
    def save_tagged(self):
        if not self.verse_data:
            self.status_var.set("No verse data to save")
            return
        
        # Update the current verse in the all_verses list
        if self.all_verses and 0 <= self.current_verse_index < len(self.all_verses):
            self.all_verses[self.current_verse_index] = self.verse_data
        
        # Save the entire JSON file
        if self.gita_json_path:
            save_path = filedialog.asksaveasfilename(
                defaultextension=".json",
                filetypes=[("JSON Files", "*.json")],
                initialfile=os.path.basename(self.gita_json_path)
            )
            
            if save_path:
                try:
                    with open(save_path, 'w', encoding='utf-8') as f:
                        json.dump(self.all_verses, f, ensure_ascii=False, indent=4)
                    self.status_var.set(f"Successfully saved tagged data to {save_path}")
                except Exception as e:
                    self.status_var.set(f"Error saving data: {str(e)}")
        else:
            # Just save the current verse
            save_path = filedialog.asksaveasfilename(
                defaultextension=".json",
                filetypes=[("JSON Files", "*.json")],
                initialfile=f"bhagavad_gita_{self.verse_data.get('chapter', '')}_{self.verse_data.get('shloka', '')}.json"
            )
            
            if save_path:
                try:
                    with open(save_path, 'w', encoding='utf-8') as f:
                        json.dump(self.verse_data, f, ensure_ascii=False, indent=4)
                    self.status_var.set(f"Successfully saved verse data to {save_path}")
                except Exception as e:
                    self.status_var.set(f"Error saving data: {str(e)}")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = GitaManualTagger(root)
    root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Playing audio
Starting playback from beginning
Pausing audio
Pause command sent to pygame
Playing audio
Starting playback from beginning
Pausing audio
Pause command sent to pygame
Playing audio
Unpausing at position 255ms
Playing audio
Unpausing at position 3633ms
Playing audio
Unpausing at position 2020ms


In [2]:
!pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [20]:
import json
import speech_recognition as sr
import librosa
import numpy as np
from pydub import AudioSegment
from fuzzywuzzy import fuzz
import os

class GitaAudioTagger:
    def __init__(self, audio_file, verse_data):
        """
        Initialize the tagger with audio file and verse data
        
        Parameters:
        - audio_file: path to the audio file of the recitation
        - verse_data: JSON data containing the verse information
        """
        self.audio_file = audio_file
        self.verse_data = verse_data
        self.audio = AudioSegment.from_file(audio_file)
        self.duration_ms = len(self.audio)
        
        # Load audio for processing
        self.y, self.sr = librosa.load(audio_file)
        
        # Extract Sanskrit words to look for in audio
        self.sanskrit_words = list(verse_data["synonyms"].keys())
        self.tagged_result = self.verse_data.copy()
        
        # Initialize timestamps for each word
        for word in self.sanskrit_words:
            self.tagged_result["synonyms"][word]["timestamp"] = {"start": None, "end": None}
    
    def detect_silence(self, min_silence_len=500, silence_thresh=-40):
        """Detect silence in audio to segment it into potential word boundaries"""
        silence_ranges = []
        current_audio = self.audio
        
        # Find silence ranges
        silence_ranges = librosa.effects.split(
            self.y, 
            top_db=abs(silence_thresh),
            frame_length=2048,
            hop_length=512
        )
        
        # Convert to milliseconds
        silence_ranges_ms = [(int(start * 1000 / self.sr), int(end * 1000 / self.sr)) 
                             for start, end in silence_ranges]
        
        return silence_ranges_ms
    
    def transcribe_segment(self, start_ms, end_ms):
        """Transcribe a specific segment of the audio"""
        segment = self.audio[start_ms:end_ms]
        temp_file = "temp_segment.wav"
        segment.export(temp_file, format="wav")
        
        recognizer = sr.Recognizer()
        with sr.AudioFile(temp_file) as source:
            audio_data = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_data, language="sa-IN")  # Sanskrit language
                return text
            except:
                return ""
        
    def match_word_to_segment(self, word, segment_text):
        """Match a Sanskrit word to transcribed segment using fuzzy matching"""
        return fuzz.ratio(word.lower(), segment_text.lower())
    
    def process_audio(self):
        """Process the audio file and tag each Sanskrit word with timestamps"""
        # Get segments based on silence
        segments = self.detect_silence()
        
        # For each segment, try to match with Sanskrit words
        for i, (start, end) in enumerate(segments):
            # Skip very short segments
            if end - start < 200:  # Less than 200ms
                continue
                
            # Transcribe the segment
            transcribed_text = self.transcribe_segment(start, end)
            
            if not transcribed_text:
                continue
                
            # Check if this segment matches any of our Sanskrit words
            for word in self.sanskrit_words:
                match_score = self.match_word_to_segment(word, transcribed_text)
                
                # If good match (threshold can be adjusted)
                if match_score > 70:  
                    # If this word hasn't been tagged yet or this is a better match
                    if (self.tagged_result["synonyms"][word]["timestamp"]["start"] is None or
                        match_score > self.tagged_result["synonyms"][word].get("match_score", 0)):
                        
                        self.tagged_result["synonyms"][word]["timestamp"]["start"] = start
                        self.tagged_result["synonyms"][word]["timestamp"]["end"] = end
                        self.tagged_result["synonyms"][word]["match_score"] = match_score
        
        # Clean up temporary match scores
        for word in self.sanskrit_words:
            if "match_score" in self.tagged_result["synonyms"][word]:
                del self.tagged_result["synonyms"][word]["match_score"]
        
        return self.tagged_result
    
    def manual_tag_word(self, word, start_ms, end_ms):
        """Manually tag a word with specific timestamps"""
        if word in self.sanskrit_words:
            self.tagged_result["synonyms"][word]["timestamp"]["start"] = start_ms
            self.tagged_result["synonyms"][word]["timestamp"]["end"] = end_ms
            return True
        return False
    
    def export_tagged_data(self, output_file):
        """Export the tagged data to a JSON file"""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.tagged_result, f, ensure_ascii=False, indent=4)
        
        print(f"Tagged data exported to {output_file}")


# Example usage
if __name__ == "__main__":
    # Load verse data from JSON
    verse_data = {
        "chapter": "2",
        "shloka": "44",
        "sanskrit": "भोगैश्वर्यप्रसक्तानां तयापहृतचेतसाम् ।\nव्यवसायात्मिका बुद्धिः समाधौ न विधीयते ॥ ४४ ॥",
        "english": "bhogaiśvarya-prasaktānāṁ\ntayāpahṛta-cetasām\nvyavasāyātmikā buddhiḥ\nsamādhau na vidhīyate",
        "synonyms": {
            "bhoga": {
                "meaning": "to material enjoyment",
                "versetext": "bhoga"
            },
            "aiśvarya": {
                "meaning": "and opulence",
                "versetext": "aiśvarya"
            },
            # ... other synonyms
        },
        "translation": "In the minds of those who are too attached to sense enjoyment and material opulence...",
        "purport": "Samādhi means \"fixed mind.\" The Vedic dictionary, the Nirukti, says..."
    }
    
    # Path to your audio file
    audio_file = "path_to_your_recitation.mp3"
    
    # Create tagger and process
    tagger = GitaAudioTagger(audio_file, verse_data)
    tagged_data = tagger.process_audio()
    
    # Export the result
    tagger.export_tagged_data("tagged_verse_2_44.json")


ModuleNotFoundError: No module named 'speech_recognition'

In [ ]:
from yt_dlp import YoutubeDL
import tkinter as tk
from tkinter import filedialog
import json
import pygame
import time

class GitaManualTagger:
    def __init__(self, root):
        self.root = root
        self.root.title("Gita Audio Tagger")
        self.root.geometry("800x600")
        
        # Initialize pygame for audio
        pygame.init()
        pygame.mixer.init()
        
        self.audio_file = None
        self.verse_data = None
        self.current_word = None
        self.playing = False
        self.current_position = 0
        
        # Setup UI
        self.setup_ui()
    
    def setup_ui(self):
        # Top frame for file selection
        top_frame = tk.Frame(self.root)
        top_frame.pack(fill=tk.X, padx=10, pady=10)
        
        tk.Button(top_frame, text="Load Audio", command=self.load_audio).pack(side=tk.LEFT, padx=5)
        tk.Button(top_frame, text="Load Verse JSON", command=self.load_verse).pack(side=tk.LEFT, padx=5)
        tk.Button(top_frame, text="Save Tagged JSON", command=self.save_tagged).pack(side=tk.LEFT, padx=5)
        
        # Middle frame for audio controls
        mid_frame = tk.Frame(self.root)
        mid_frame.pack(fill=tk.X, padx=10, pady=10)
        
        tk.Button(mid_frame, text="Play/Pause", command=self.toggle_play).pack(side=tk.LEFT, padx=5)
        tk.Button(mid_frame, text="Mark Start", command=lambda: self.mark_timestamp("start")).pack(side=tk.LEFT, padx=5)
        tk.Button(mid_frame, text="Mark End", command=lambda: self.mark_timestamp("end")).pack(side=tk.LEFT, padx=5)
        
        self.position_label = tk.Label(mid_frame, text="Position: 0:00")
        self.position_label.pack(side=tk.LEFT, padx=20)
        
        # Frame for word selection
        word_frame = tk.Frame(self.root)
        word_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        tk.Label(word_frame, text="Select Word to Tag:").pack(anchor=tk.W)
        
        self.word_listbox = tk.Listbox(word_frame)
        self.word_listbox.pack(fill=tk.BOTH, expand=True)
        self.word_listbox.bind('<<ListboxSelect>>', self.on_word_select)
        
        # Frame for word details
        details_frame = tk.Frame(self.root)
        details_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.word_details = tk.Text(details_frame, height=5)
        self.word_details.pack(fill=tk.X)
        
        # Status bar
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        status_bar = tk.Label(self.root, textvariable=self.status_var, bd=1, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Start position update timer
        self.update_position()
    
    def load_audio(self):
        self.audio_file = filedialog.askopenfilename(filetypes=[("Audio Files", "*.mp3;*.wav")])
        if self.audio_file:
            pygame.mixer.music.load(self.audio_file)
            self.status_var.set(f"Loaded audio: {self.audio_file}")
    
    def load_verse(self):
        json_file = filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])
        if json_file:
            with open(json_file, 'r', encoding='utf-8') as f:
                self.verse_data = json.load(f)
            
            # Populate word list
            self.word_listbox.delete(0, tk.END)
            for word in self.verse_data["synonyms"]:
                self.word_listbox.insert(tk.END, word)
            
            self.status_var.set(f"Loaded verse: Chapter {self.verse_data['chapter']}, Shloka {self.verse_data['shloka']}")
    def toggle_play(self):
        if not self.audio_file:
            self.status_var.set("No audio file loaded")
            return
            
        if self.playing:
            pygame.mixer.music.pause()
            self.playing = False
        else:
            if pygame.mixer.music.get_pos() == -1:  # Not started yet
                pygame.mixer.music.play()
            else:
                pygame.mixer.music.unpause()
            self.playing = True
    
    def update_position(self):
        if self.playing and pygame.mixer.music.get_busy():
            pos_ms = pygame.mixer.music.get_pos()
            if pos_ms >= 0:
                self.current_position = pos_ms
                minutes = pos_ms // 60000
                seconds = (pos_ms % 60000) // 1000
                self.position_label.config(text=f"Position: {minutes}:{seconds:02d}")
        
        # Schedule next update
        self.root.after(100, self.update_position)
    
    def on_word_select(self, event):
        if not self.verse_data:
            return
            
        selection = self.word_listbox.curselection()
        if selection:
            self.current_word = self.word_listbox.get(selection[0])
            word_data = self.verse_data["synonyms"][self.current_word]
            
            # Update word details
            details = f"Word: {self.current_word}\n"
            details += f"Meaning: {word_data['meaning']}\n"
            details += f"Verse Text: {word_data['versetext']}\n"
            
            # Add timestamp info if available
            if "timestamp" in word_data:
                start = word_data["timestamp"].get("start")
                end = word_data["timestamp"].get("end")
                details += f"Start: {start} ms, End: {end} ms"
            
            self.word_details.delete(1.0, tk.END)
            self.word_details.insert(tk.END, details)
    
    def mark_timestamp(self, mark_type):
        if not self.current_word or not self.verse_data:
            self.status_var.set("No word selected or verse data loaded")
            return
        
        # Ensure timestamp field exists
        if "timestamp" not in self.verse_data["synonyms"][self.current_word]:
            self.verse_data["synonyms"][self.current_word]["timestamp"] = {"start": None, "end": None}
        
        # Set the timestamp
        self.verse_data["synonyms"][self.current_word]["timestamp"][mark_type] = self.current_position
        
        # Update display
        self.on_word_select(None)  # Refresh details
        self.status_var.set(f"Marked {mark_type} timestamp for '{self.current_word}' at {self.current_position} ms")
    
    def save_tagged(self):
        if not self.verse_data:
            self.status_var.set("No verse data to save")
            return
            
        save_file = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON Files", "*.json")])
        if save_file:
            with open(save_file, 'w', encoding='utf-8') as f:
                json.dump(self.verse_data, f, ensure_ascii=False, indent=4)
            self.status_var.set(f"Tagged data saved to {save_file}")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = GitaManualTagger(root)
    root.mainloop()


[youtube:tab] Extracting URL: https://www.youtube.com/@BrajaBeats/shorts
[youtube:tab] @BrajaBeats/shorts: Downloading webpage
[download] Downloading playlist: Braja Beats - Shorts
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 1: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 2: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 3: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 4: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 5: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 6: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 7: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 8: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 9: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 10: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 11: Downloading API JSON
[youtube:tab] UCm9yeAhuccxsL-mtCrRTqmQ page 12: Downloading API JSON


[info] oiJ9YjYMNEw: Downloading 1 format(s): 251
[download] Not a Bhagavad-gita video
[download] Downloading item 34 of 638
[youtube] Extracting URL: https://www.youtube.com/shorts/PQtu3ang2NA
[youtube] PQtu3ang2NA: Downloading webpage


[youtube] PQtu3ang2NA: Downloading webpage
[youtube] PQtu3ang2NA: Downloading tv client config
[youtube] PQtu3ang2NA: Downloading tv player API JSON
[youtube] PQtu3ang2NA: Downloading ios player API JSON
[youtube] PQtu3ang2NA: Downloading m3u8 information
[info] PQtu3ang2NA: Downloading 1 format(s): 251
[download] Destination: Bhagavad-gita 8.27#shorts #mindfulness #mantra meditation #gita #mantra chanting [PQtu3ang2NA].webm
[download] 100% of  962.02KiB in 00:00:00 at 1.72MiB/s     
[ExtractAudio] Destination: Bhagavad-gita 8.27#shorts #mindfulness #mantra meditation #gita #mantra chanting [PQtu3ang2NA].mp3
Deleting original file Bhagavad-gita 8.27#shorts #mindfulness #mantra meditation #gita #mantra chanting [PQtu3ang2NA].webm (pass -k to keep)
[download] Downloading item 35 of 638
[youtube] Extracting URL: https://www.youtube.com/shorts/OSUF88h1XwE
[youtube] OSUF88h1XwE: Downloading webpage
[youtube] OSUF88h1XwE: Downloading tv client config
[youtube] OSUF88h1XwE: Downloading tv pla

[download] Got error: <urllib3.connection.HTTPSConnection object at 0x79579bcfbeb0>: Failed to establish a new connection: [Errno 101] Network is unreachable


DownloadError: ERROR: [download] Got error: <urllib3.connection.HTTPSConnection object at 0x79579bcfbeb0>: Failed to establish a new connection: [Errno 101] Network is unreachable

In [ ]:

# Configure yt-dlp to extract only shorts
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'match_filter': lambda info: None if 'Bhagavad-gita' in info.get('title', '') else 'Not a Bhagavad-gita video',
    'playlistrandom': True,  # To get all videos
}

# Get shorts from the channel
with YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/@BrajaBeats/shorts'])



Completed! 0 files were renamed.


# Waveform Audio Tagger

Run the new waveform-based audio tagger for easier audio segment selection.

In [2]:
import os
import sys

# Get the current directory
current_dir = os.getcwd()

# Add the path to sys.path if it's not already there
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import and run the waveform tagger
try:
    from waveform_tagger import GitaWaveformTagger
    import tkinter as tk
    
    # Check if audio directory exists
    audio_dir = os.path.join(current_dir, "BrajaBeats_Gita_MP3")
    if not os.path.exists(audio_dir):
        print(f"Warning: Audio directory not found at {audio_dir}")
        print("Please run the audio download cell first or check the directory path.")
    else:
        print(f"Found audio directory at {audio_dir} with {len(os.listdir(audio_dir))} files")
    
    # Check if gita.json exists
    gita_json_path = os.path.join(current_dir, "gita.json")
    if not os.path.exists(gita_json_path):
        print(f"Warning: gita.json not found at {gita_json_path}")
        print("Please make sure to have the Gita JSON file for verse data.")
    else:
        print(f"Found gita.json at {gita_json_path}")
    
    # Start the waveform tagger
    print("\nStarting the Waveform Tagger application...")
    root = tk.Tk()
    app = GitaWaveformTagger(root)
    root.mainloop()
except ImportError as e:
    print(f"Error importing the waveform tagger: {e}")
    print("\nMake sure the file 'waveform_tagger.py' is in the current directory.")
except Exception as e:
    print(f"Error running the waveform tagger: {e}")


Found audio directory at /home/manideepdonkena/Documents/SPRITUAL/audio_download/BrajaBeats_Gita_MP3 with 103 files
Found gita.json at /home/manideepdonkena/Documents/SPRITUAL/audio_download/gita.json

Starting the Waveform Tagger application...


invalid command name "136290256829696update_position"
    while executing
"136290256829696update_position"
    ("after" script)


# File System Inspection

Use these cells to check what files are available in the current directory.

In [1]:

!pip install --upgrade google-cloud-speech

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/334.1 KB ? eta -:--:--  Downloading google_cloud_speech-2.32.0-py3-none-any.whl (334 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 KB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 KB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 KB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 KB 42.3 MB/s eta 0:00:00
  Using cached google_api_core-2.24.2-py3-none-any.whl (160 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl (160 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.9/320.9 KB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.9/320.9 KB 42.1 MB/s eta 0:00:00
  Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 KB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from google.cloud import speech

audio_uri = "BrajaBeats_Gita_MP3/Bhagavad-gita 13.6.mp3"
def transcribe_gcs_with_word_time_offsets():


    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(uri=audio_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code="en-US",
        enable_word_time_offsets=True,
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    result = operation.result(timeout=90)

    for result in result.results:
        alternative = result.alternatives[0]
        print(f"Transcript: {alternative.transcript}")
        print(f"Confidence: {alternative.confidence}")

        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time

            print(
                f"Word: {word}, start_time: {start_time.total_seconds()}, end_time: {end_time.total_seconds()}"
            )

    return result

transcribe_gcs_with_word_time_offsets()


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [5]:
# List files in the current directory
import os

current_dir = os.getcwd()
print(f"Current directory: {current_dir}")
print("\nFiles in current directory:")
for item in os.listdir(current_dir):
    print(f" - {item}")

# Check audio directory contents
audio_dir = os.path.join(current_dir, "BrajaBeats_Gita_MP3")
if os.path.exists(audio_dir):
    print(f"\nFiles in audio directory ({len(os.listdir(audio_dir))} files):")
    for i, item in enumerate(sorted(os.listdir(audio_dir))):
        if i < 20:  # Show only first 20 files to avoid clutter
            print(f" - {item}")
        elif i == 20:
            print(f" - ... and {len(os.listdir(audio_dir)) - 20} more files")
else:
    print(f"\nAudio directory not found at {audio_dir}")


Current directory: /home/manideepdonkena/Documents/SPRITUAL/audio_download

Files in current directory:
 - __pycache__
 - tags.json
 - chage_name.py
 - waveform_tagger.py
 - .vscode
 - BrajaBeats_Gita_MP3
 - gita.json
 - downloader.ipynb

Files in audio directory (103 files):
 - Bhagavad-gita 1.14.mp3
 - Bhagavad-gita 1.16.mp3
 - Bhagavad-gita 1.31.mp3
 - Bhagavad-gita 1.32.mp3
 - Bhagavad-gita 1.33.mp3
 - Bhagavad-gita 1.34.mp3
 - Bhagavad-gita 1.38.mp3
 - Bhagavad-gita 1.40.mp3
 - Bhagavad-gita 1.43.mp3
 - Bhagavad-gita 1.6.mp3
 - Bhagavad-gita 13.1.mp3
 - Bhagavad-gita 13.15.mp3
 - Bhagavad-gita 13.23.mp3
 - Bhagavad-gita 13.26.mp3
 - Bhagavad-gita 13.6.mp3
 - Bhagavad-gita 13.9.mp3
 - Bhagavad-gita 14.10.mp3
 - Bhagavad-gita 14.14.mp3
 - Bhagavad-gita 14.17.mp3
 - Bhagavad-gita 14.18.mp3
 - ... and 83 more files


In [6]:
# Check file permissions and fix if needed
import os
import stat

def fix_permissions(directory):
    try:
        # Fix directory permissions
        os.chmod(directory, stat.S_IRWXU | stat.S_IRWXG | stat.S_IROTH | stat.S_IXOTH)
        print(f"Fixed permissions for directory: {directory}")
        
        # Fix file permissions inside the directory
        if os.path.exists(directory) and os.path.isdir(directory):
            for item in os.listdir(directory):
                item_path = os.path.join(directory, item)
                if os.path.isfile(item_path):
                    os.chmod(item_path, stat.S_IRUSR | stat.S_IWUSR | stat.S_IRGRP | stat.S_IWGRP | stat.S_IROTH)
                    print(f"Fixed permissions for file: {item_path}")
        return True
    except Exception as e:
        print(f"Error fixing permissions: {e}")
        return False

# Fix permissions for current directory and audio directory
current_dir = os.getcwd()
audio_dir = os.path.join(current_dir, "BrajaBeats_Gita_MP3")

print("Fixing permissions for current directory...")
fix_permissions(current_dir)

if os.path.exists(audio_dir):
    print("\nFixing permissions for audio directory...")
    fix_permissions(audio_dir)
else:
    print(f"\nAudio directory not found at {audio_dir}")


Fixing permissions for current directory...
Fixed permissions for directory: /home/manideepdonkena/Documents/SPRITUAL/audio_download
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/tags.json
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/chage_name.py
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/waveform_tagger.py
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/gita.json
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/downloader.ipynb

Fixing permissions for audio directory...
Fixed permissions for directory: /home/manideepdonkena/Documents/SPRITUAL/audio_download/BrajaBeats_Gita_MP3
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/BrajaBeats_Gita_MP3/Bhagavad-gita 9.20.mp3
Fixed permissions for file: /home/manideepdonkena/Documents/SPRITUAL/audio_download/BrajaBeats_Git